In [1]:
!pip install  --upgrade "transformers"   "datasets"  "accelerate"  "evaluate"  "bitsandbytes"  "trl"  "peft" 
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 742.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments
import torch
import os

2024-09-02 07:38:51.580418: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-02 07:38:51.580541: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-02 07:38:51.705778: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from huggingface_hub import login

login(
  token="fill ur token",
  add_to_git_credential=True
)

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
# csv loading
csv_file_path = "/kaggle/input/llama-fine-tuning-v1/fine-tuning-llama-v1_order_values_train.csv" 
df = pd.read_csv(csv_file_path)
print(df.columns)
print(len(df))
print(df.head())
dataset = Dataset.from_pandas(df)

Index(['question', 'ground_truth_query', 'execution_status_gtu',
       'execution_result_gtu', 'llama_sqlcoder_3_few_promp',
       'execution_status_llama', 'execution_result_llama'],
      dtype='object')
54
                                            question  \
0       Show me trend of sales for the last 6 months   
1  Show me a quarterly trend of sales for the las...   
2  Trend of average order value AOV for the last ...   
3           Monthly trend of average order value AOV   
4                Net revenue by state for last month   

                                  ground_truth_query execution_status_gtu  \
0  SELECT CONCAT(order_year, '-', LPAD(order_mont...              Success   
1  SELECT CONCAT(order_year, '-Q', order_quarter)...              Success   
2  SELECT     CONCAT(order_year, '-', LPAD(order_...              Success   
3  WITH order_totals AS (SELECT order_id, order_y...              Success   
4  SELECT LOWER(billing_address_state) AS state, ...              S

In [5]:
database_type = "databricks-db"
database_schema = """{
    "order_values": "This table contains comprehensive order information including pricing, tax, discounts, returns, shipping, and product details. Each row represents a unique line item within an order and includes timestamps, statuses, and financials associated with the order.",
}
COLUMNS = {
    "order_values": {
        "order_line_item_id": {
            "description": "Unique identifier for each line item in an order",
            "values_sample": ["shopify_5667598663969_14461737632033"],
        },
        "source": {
            "description": "Source of the order (e.g., platform or channel)",
            "values_sample": ["shopify"],
        },
        "order_id": {
            "description": "Unique identifier for each order",
            "values_sample": ["shopify_5667598663969"],
        },
        "ordered_quantity": {
            "description": "Quantity of each item ordered",
            "values_sample": [1],
        },
        "gross_sales_price_after_tax": {
            "description": "Gross sales price after tax is applied",
            "values_sample": [150.00],
        },
        "tax_percent": {
            "description": "Tax percentage applied to the order",
            "values_sample": [18.0],
        },
        "total_price_tax": {
            "description": "Total tax amount applied to the order",
            "values_sample": [20.00],
        },
        "gross_sales_price_before_tax": {
            "description": "Gross sales price before tax is applied",
            "values_sample": [127.12],
        },
        "total_discount_after_tax": {
            "description": "Total discount amount after tax is applied",
            "values_sample": [10.00],
        },
        "total_discount_tax": {
            "description": "Total tax amount on the discount applied",
            "values_sample": [1.80],
        },
        "total_discount_before_tax": {
            "description": "Total discount amount before tax is applied",
            "values_sample": [8.20],
        },
        "total_cancellation_returns_before_tax": {
            "description": "Total cancellation returns amount before tax",
            "values_sample": [20.00],
        },
        "total_rto_returns_before_tax": {
            "description": "Total RTO (Return to Origin) returns amount before tax",
            "values_sample": [15.00],
        },
        "total_returns_refund_before_tax": {
            "description": "Total refund amount before tax due to returns",
            "values_sample": [30.00],
        },
        "total_returns_before_tax": {
            "description": "Total returns amount before tax, including all types of returns",
            "values_sample": [25.00],
        },
        "total_return_tax": {
            "description": "Total tax amount on returns",
            "values_sample": [4.50],
        },
        "net_sales_before_tax": {
            "description": "Net sales amount before tax is applied",
            "values_sample": [100.00],
        },
        "item_shipping_charge": {
            "description": "Shipping charge for the item",
            "values_sample": [5.00],
        },
        "item_duties": {
            "description": "Duties charged for the item",
            "values_sample": [3.00],
        },
        "item_additional_fees": {
            "description": "Additional fees associated with the item",
            "values_sample": [2.00],
        },
        "total_tax": {
            "description": "Total tax amount applied to the order",
            "values_sample": [18.00],
        },
        "total_sales_after_tax": {
            "description": "Total sales amount after tax is applied",
            "values_sample": [120.00],
        },
        "gross_merchandise_value": {
            "description": "Gross merchandise value of the item",
            "values_sample": [150.00],
        },
        "sku_id": {
            "description": "Unique identifier for each SKU (Stock Keeping Unit)",
            "values_sample": ["shopify_8848983261473_47478863036705"],
        },
        "sku_code": {
            "description": "Code assigned to each SKU",
            "values_sample": ["BSL-D-SIP-0001"],
        },
        "product_category": {
            "description": "Category of the product",
            "values_sample": ["Insulated Sipper"],
        },
        "product_id": {
            "description": "Unique identifier for each product",
            "values_sample": ["shopify_8848983261473"],
        },
        "product_title": {
            "description": "Title of the product",
            "values_sample": ["Kids Sipper (430 ml)"],
        },
        "product_price": {
            "description": "Price of the product",
            "values_sample": [999],
        },
        "customer_id": {
            "description": "Unique identifier for each customer",
            "values_sample": ["shopify_7694075199777"],
        },
        "first_ordered_at": {
            "description": "Timestamp of the customer's first order",
            "values_sample": ["2024-02-14 16:00:13.000000"],
        },
        "order_refund_line_item_id": {
            "description": "Unique identifier for each refunded line item in an order",
            "values_sample": ["shopify_5667598663969_14461737632033"],
        },
        "order_date_time_utc": {
            "description": "Order date and time in UTC",
            "values_sample": ["2024-02-14 16:00:13.000000"],
        },
        "order_date_time": {
            "description": "Local date and time of the order",
            "values_sample": ["2024-02-14 11:00:13.000000"],
        },
        "order_date": {
            "description": "Date of the order",
            "values_sample": ["2024-02-14"],
        },
        "order_year": {
            "description": "Year of the order",
            "values_sample": [2024],
        },
        "order_quarter": {
            "description": "Quarter of the year when the order was placed",
            "values_sample": [1],
        },
        "order_month": {
            "description": "Month of the order",
            "values_sample": [2],
        },
        "order_day_of_month": {
            "description": "Day of the month of the order",
            "values_sample": [14],
        },
        "order_week": {
            "description": "Week of the year when the order was placed",
            "values_sample": [7],
        },
        "order_day_of_week": {
            "description": "Day of the week of the order",
            "values_sample": [4],  # Assuming Sunday is 0
        },
        "order_hour": {
            "description": "Hour of the day when the order was placed",
            "values_sample": [16],
        },
        "order_minute": {
            "description": "Minute of the hour when the order was placed",
            "values_sample": [0],
        },
        "order_time": {
            "description": "Time of the order",
            "values_sample": ["16:00"],
        },
        "refund_status": {
            "description": "Refund status of the order",
            "values_sample": ["false"],
        },
        "rto_status": {
            "description": "Return to Origin (RTO) status of the order",
            "values_sample": ["false"],
        },
        "cancellation_status": {
            "description": "Cancellation status of the order",
            "values_sample": ["false"],
        },
        "order_status": {
            "description": "Current status of the order",
            "values_sample": ["completed"],
        },
        "payment_status": {
            "description": "Payment status of the order",
            "values_sample": ["paid"],
        },
        "return_reason": {
            "description": "Reason for return, if applicable",
            "values_sample": ["damaged item"],
        },
        "billing_address_city": {
            "description": "City of the billing address",
            "values_sample": ["Surat"],
        },
        "billing_address_state": {
            "description": "State of the billing address",
            "values_sample": ["Gujarat"],
        },
        "billing_address_country": {
            "description": "Country of the billing address",
            "values_sample": ["India"],
        },
        "gift_wrap_expense": {
            "description": "Expense incurred for gift wrapping",
            "values_sample": [5.00],
        },
        "packaging_expense": {
            "description": "Expense incurred for packaging",
            "values_sample": [3.00],
        },
        "handling_expense": {
            "description": "Expense incurred for handling",
            "values_sample": [2.00],
        },
        "shipping_expense": {
            "description": "Expense incurred for shipping",
            "values_sample": [10.00],
        },
        "marketplace_expense": {
            "description": "Marketplace fees associated with the order",
            "values_sample": [8.00],
        },
        "payment_gateway_expense": {
            "description": "Fees charged by the payment gateway",
            "values_sample": [4.00],
        },
        "other_adjustments": {
            "description": "Other adjustments related to the order",
            "values_sample": [2.50],
        },
        "estimated_expense": {
            "description": "Estimated total expense for the order",
            "values_sample": [20.00],
        },
        "sku_cost": {
            "description": "Cost of the SKU",
            "values_sample": [75.00],
        },
        "gross_profit": {
            "description": "Gross profit for the order",
            "values_sample": [50.00],
        },
        "cm1_expense": {
            "description": "CM1 (Contribution Margin 1) expense",
            "values_sample": [10.00],
        },
        "profit_cm1": {
            "description": "CM1 profit for the order",
            "values_sample": [40.00],
        },
        "_synced_at": {
            "description": "Timestamp when the data was last synced",
            "values_sample": ["2024-02-14 16:30:00.000000"],
        },
    },
}
"""


In [6]:

system_message = """
You are a SQL expert in Mysql. As a SQL expert, your job is to answer data related questions from the business team accurately by writing SQL queries. Refer to the provided database schema for tables and columns. Do not correct spelling mistakes in the schema.

Please generate only read/SELECT queries for data safety. If a question cannot be answered using a SELECT operation, it should not be addressed.
Refer the sample queries to see how date filters and calculations are done. Pay high attention on sample queries that seem most relevant to user question.

Format your responses as JSON, including the SQL query and the following associated fields. Use this format:

generated_query:[{{"sql_query": "Your SQL Query Here", "logic": "Explain briefly the logic you used", "response": "A friendly and conversational response to give more context to the user.", "chart": {{"type": "The type of chart", "x_axis": ["Array of variables in the x-axis of the chart"], "y_axis": ["Array of variables in the y-axis of the chart"]}} }}]

## Database Schema:
{}
## Guidelines that are necessary to follow for making correct SQL queries:
1. Whenever working with any text fields, always cast both column and value to lowercase before performing any operations on them.
2. Chart Type, X-axis and Y-axis will be "null" when the question asks for a single value instead of a trend or split.
3. Ensure numeric typecasting for numerators in division operations.
4. Use lowercase for string matching to maintain consistency. Note: Do type casting to varchar only if needed (mainly in case of Mssql DB type).
5. Apply the same operation in both GROUP BY and SELECT clauses when referencing the same field.
6. Create aliases for fields in SELECT on which operations are applied.
7. Always prefix field names with table or alias names in joins.
8. Treat numeric fields as measures and text, enums, and time fields as dimensions in aggregation queries.
9. Please do not refer the database name in column or table names.
10. Prevent zero division errors by validating the denominator in division operations.
11. Exclude null values in aggregations of numeric fields.
12. Use LOWER in GROUP BY only if the same column in SELECT is also in LOWER.
13. Please consider the granularity level of the table and do appropriate aggregations.
e.g.: If order_data table has Order ID - SKU ID columns, and question is asked at order level, group rows by Order ID. Write queries accordingly.
14. In case time period is not mentioned, use last 12 calendar months data only for calculations, unless lifetime value like questions are asked.
15. Consider order by ABS value wherever seems logical.
16. Round the calculations to 2 decimal points at appropriate places, like AVG and Percentages.
17. Please type-cast the date columns appropriately before making any date based comparisons (only if needed). E.g. for IBM I DB2 ``` to_date(cast(date_column as char(8)),'YYYYMMDD') < current date' ```

## Additional guidelines specific for IBM I DB2:
18. In case of IBM I DB2 queries do not end with semi-colon ';'
19. In case of IBM I DB2 syntax to get top x rows is 'FETCH TOP x ROWS ONLY'.
20. Column names in IBM I DB2 are in upper case and should match in chart dict.

## Response:
""".format(database_schema)


In [7]:
len(database_schema.split())

856

In [8]:
len(system_message.split())

1398

In [9]:
# create conversation chain 
def create_conversation(sample):
    return {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": sample["question"]},
            {"role": "assistant", "content": sample["ground_truth_query"]}
        ]
    }

In [10]:
dataset = dataset.map(create_conversation)

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [11]:
test_size = 2
train_size = len(dataset) - test_size

In [12]:
dataset = dataset.shuffle(seed=42) 
test_dataset = dataset.select(range(test_size))
train_dataset = dataset.select(range(test_size, test_size + train_size))

In [13]:
save_dir = "llama3-8b-instruct-text-to-sql-order_values"
os.makedirs(save_dir, exist_ok=True)

In [14]:
test_dataset.to_json(os.path.join(save_dir, "test_dataset.json"), orient="records")
train_dataset.to_json(os.path.join(save_dir, "train_dataset.json"), orient="records")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

950821

In [15]:
#Data setLoading
dataset = DatasetDict({
    "train": load_dataset("json", data_files=os.path.join(save_dir, "train_dataset.json"), split="train"),
    "test": load_dataset("json", data_files=os.path.join(save_dir, "test_dataset.json"), split="train")
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
#model name
model_id = "defog/llama-3-sqlcoder-8b"
torch_dtype = torch.float16

In [17]:
# quantization for memory optimization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype
)

In [18]:
#model loading
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch_dtype,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [19]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [20]:
#lora configurations
peft_config = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

In [21]:
# training arguments and training
args = TrainingArguments(
    output_dir=save_dir,
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
    report_to="tensorboard",
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=1600,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)
trainer.train()


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, packing, dataset_kwargs. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:195: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:327: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the o

Generating train split: 0 examples [00:00, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,0.689500
20,0.089100
30,0.047700
40,0.088100
50,0.055900
60,0.075900
70,0.061000
80,0.050800
90,0.037200
100,0.056300


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=200, training_loss=0.08126032158732414, metrics={'train_runtime': 4840.7457, 'train_samples_per_second': 0.041, 'train_steps_per_second': 0.041, 'total_flos': 1.448998600704e+16, 'train_loss': 0.08126032158732414, 'epoch': 2.0})

In [22]:
trainer.save_model(output_dir=save_dir)  
tokenizer.save_pretrained(save_dir)  

('llama3-8b-instruct-text-to-sql-order_values/tokenizer_config.json',
 'llama3-8b-instruct-text-to-sql-order_values/special_tokens_map.json',
 'llama3-8b-instruct-text-to-sql-order_values/tokenizer.json')

In [23]:
del model
del trainer
torch.cuda.empty_cache()